In [50]:
from pylab import *
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import folium

In [28]:
data_path=  "C:/Travail/Central Digital Lab/Data Challenge/train.csv"
data_sample = pd.read_csv(data_path,skiprows = range(1,54900000))
#data_sample.columns = ["key","fare","pickup_datetime","pickup_longitude",
                       #"pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"]

In [54]:
data_sample.iloc[1]['pickup_latitude']

40.750713

In [55]:
data = pd.DataFrame({
'lat': data_sample['pickup_latitude'][0:50],
'lon': data_sample['pickup_longitude'][0:50],
})
data

m = folium.Map(location=(40.7648,-73.9808),tiles='Stamen Terrain',zoom_start = 12)

for i in range(0,len(data)):
    folium.Marker([data.iloc[i]['lat'],data.iloc[i]['lon']]).add_to(m)
    
#m.save('./')

In [56]:
m

In [39]:
X_taxi = data_sample.values[:,2:]
Y_taxi = data_sample.values[:,1]

array([['2011-08-31 07:05:00 UTC', -73.98151700000001, 40.741165,
        -73.977287, 40.754762, 2],
       ['2013-09-24 16:19:02 UTC', -74.002, 40.750713, -73.964212,
        40.773633000000004, 2],
       ['2014-09-09 07:26:00 UTC', -73.99765, 40.683425,
        -73.95853699999999, 40.760137, 1],
       ...,
       ['2011-04-02 22:04:24 UTC', -73.970505, 40.752325, -73.960537,
        40.797342, 1],
       ['2011-10-26 05:57:51 UTC', -73.980901, 40.764629, -73.870605,
        40.773963, 1],
       ['2014-12-12 11:33:00 UTC', -73.96972199999999, 40.797668,
        -73.970885, 40.783313, 1]], dtype=object)

In [ ]:
plt.plot(X_taxi[:,])

In [29]:
data_sample.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-08-31 07:05:00.00000067,6.1,2011-08-31 07:05:00 UTC,-73.981517,40.741165,-73.977287,40.754762,2
1,2013-09-24 16:19:02.0000006,17.0,2013-09-24 16:19:02 UTC,-74.002000,40.750713,-73.964212,40.773633,2
2,2014-09-09 07:26:00.000000153,24.5,2014-09-09 07:26:00 UTC,-73.997650,40.683425,-73.958537,40.760137,1
3,2009-10-05 14:51:33.0000001,5.7,2009-10-05 14:51:33 UTC,-74.006147,40.714288,-74.015233,40.711181,1
4,2010-05-27 22:43:00.00000097,6.5,2010-05-27 22:43:00 UTC,-73.976137,40.755605,-73.990392,40.750540,1


In [40]:
X_taxi_train,Y_taxi_train,X_taxi_train,Y_taxi_train = train_test_split(
        X_taxi,Y_taxi,test_size=0.2,random_state = 0)